In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Replace with your own database credentials
db_config = {
    "host": "db00",
    "user": "root",
    "password": "Local1234567",
    "database": "osticket_1-10-1_live"
}
# Create a SQLAlchemy engine using the MySQL connector connection
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

# Define your SQL query
query = """
    SELECT 
        CONCAT(st.firstname, ' ', st.lastname) AS 'AssignedTo',
        te.name AS AssignedToTeam,
        t.created AS created,
        t.updated AS updated,
        Title.value AS Title,
        Classification.value AS Classification,
        Topic.value AS Topic,
        tre.body AS Text
    FROM 
        ost_ticket t
        JOIN ost_ticket_status ts ON (t.status_id = ts.id)
        JOIN ost_department d ON (t.dept_id = d.id)
        JOIN ost_thread tr ON (t.ticket_id = tr.object_id)
        JOIN ost_thread_entry tre ON (tr.id = tre.thread_id AND tre.type = 'M' AND tre.pid = 0)
        JOIN ost_form_entry f1 ON (t.ticket_id = f1.object_id AND f1.form_id = 2)
        LEFT JOIN ost_staff st ON (t.staff_id = st.staff_id)
        LEFT JOIN ost_team te ON (t.team_id = te.team_id)
        LEFT JOIN ost_form_entry_values Title ON (f1.id = Title.entry_id AND Title.field_id = 5 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Priority ON (f1.id = Priority.entry_id AND Priority.field_id = 7 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Classification ON (f1.id = Classification.entry_id AND Classification.field_id = 29 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Topic ON (f1.id = Topic.entry_id AND Topic.field_id = 31 AND f1.form_id = 2)
    WHERE 
        d.name = 'TABZ FLS'
    ORDER BY 
        created DESC
    LIMIT 5000
"""

# Use SQLAlchemy engine to execute the query and load data into a DataFrame
df = pd.read_sql(query, engine)


In [2]:
print(df)

              AssignedTo            AssignedToTeam             created  \
0                   None                       NAV 2023-09-08 08:14:54   
1        Arashdeep Singh               H/S Support 2023-09-08 08:01:56   
2     Francesco Zambaldi               H/S Support 2023-09-07 17:26:32   
3     Francesco Zambaldi               H/S Support 2023-09-07 16:43:13   
4        Arashdeep Singh  User Identity Management 2023-09-07 16:29:12   
...                  ...                       ...                 ...   
4995        Daniele Neri                       NAV 2023-03-13 14:26:04   
4996        Daniele Neri                       NAV 2023-03-13 14:26:04   
4997        Daniele Neri                       NAV 2023-03-13 14:14:02   
4998     Arashdeep Singh               H/S Support 2023-03-13 14:05:14   
4999        Daniele Neri                       NAV 2023-03-13 13:42:20   

                 updated                                   Title  \
0    2023-09-08 08:14:54  Detach Purchase /

In [3]:
# List of columns to keep (exclude the ones you want to remove)
columns_to_keep = ['Title', 'Topic', 'Text']

# Create a new DataFrame with only the selected columns
df = df[columns_to_keep]

# df now contains only columns A and B
print(df)

                                       Title  \
0     Detach Purchase / Sales Order NAV TAEE   
1                                   Tastiera   
2           Outlook  Termine erstellen TEams   
3                       IDEpro Authenticator   
4            NOSAL MIROSLAV - Badge / ACCESS   
...                                      ...   
4995          Microsoft Dynamic NAV approach   
4996          Microsoft Dynamic NAV approach   
4997                Quantità ordine acquisto   
4998                     Computer più veloce   
4999                     Prodiktionsaufträge   

                                                  Topic  \
0                                 {"22":"Dynamics NAV"}   
1     {"11":"Hardware, Telephony, Smartphones, Monit...   
2     {"12":"Standard Software: Operating systems, M...   
3     {"12":"Standard Software: Operating systems, M...   
4     {"16":"User identity management, Permissions, ...   
...                                                 ...   
4995      

In [4]:
import re

values_to_keep = [
    '{"16":"User identity management, Permissions, Badge, Mail"}',
    '{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}',
    '{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}'
]

# Use str.contains to create a boolean mask, replace NaN with False, and then filter
mask = df['Topic'].str.contains('|'.join(map(re.escape, values_to_keep))).fillna(False)
filtered_df = df[mask]

# Print the filtered DataFrame
print(filtered_df)

                                 Title  \
1                             Tastiera   
2     Outlook  Termine erstellen TEams   
3                 IDEpro Authenticator   
4      NOSAL MIROSLAV - Badge / ACCESS   
5                          Licenze CRM   
...                                ...   
4980                carattere mancante   
4984    Netzwerk und Kameraeinstellung   
4990          PC di scorta per Stefano   
4991          Stampante ufficio Italia   
4998               Computer più veloce   

                                                  Topic  \
1     {"11":"Hardware, Telephony, Smartphones, Monit...   
2     {"12":"Standard Software: Operating systems, M...   
3     {"12":"Standard Software: Operating systems, M...   
4     {"16":"User identity management, Permissions, ...   
5     {"16":"User identity management, Permissions, ...   
...                                                 ...   
4980  {"12":"Standard Software: Operating systems, M...   
4984  {"12":"Standard S

In [5]:
# Reset the index to start from 0
filtered_df = filtered_df.reset_index(drop=True)

# Print the filtered DataFrame
print(filtered_df)

                                 Title  \
0                             Tastiera   
1     Outlook  Termine erstellen TEams   
2                 IDEpro Authenticator   
3      NOSAL MIROSLAV - Badge / ACCESS   
4                          Licenze CRM   
...                                ...   
2717                carattere mancante   
2718    Netzwerk und Kameraeinstellung   
2719          PC di scorta per Stefano   
2720          Stampante ufficio Italia   
2721               Computer più veloce   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"12":"Standard Software: Operating systems, M...   
3     {"16":"User identity management, Permissions, ...   
4     {"16":"User identity management, Permissions, ...   
...                                                 ...   
2717  {"12":"Standard Software: Operating systems, M...   
2718  {"12":"Standard S

In [6]:
from IPython.display import display, HTML

display(HTML('<style>.output { overflow-x: scroll; white-space: nowrap; }</style>'))
filtered_df

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...","<p>Ciao,</p> <p>la mia tastiera purtropppo non..."
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...","<p>Ciao,</p> <p>da oggi non vedo piu il link p..."
2,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...","<p>Ciao,</p> <p>Ho bisogno di accedere a IDEpr..."
3,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...","Ciao,<br />lui deve acceder all'appartamento A..."
4,Licenze CRM,"{""16"":""User identity management, Permissions, ...","<p>Ciao,<br />purtroppo hanno fatto un pò casi..."
...,...,...,...
2717,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",<p>ora funziona grazie mille</p>
2718,Netzwerk und Kameraeinstellung,"{""12"":""Standard Software: Operating systems, M...","<p>Hallo, </p> <p><br /></p> <p>ich habe keine..."
2719,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...","Ciao,<br /><br />prego dare a Stefano un PC di..."
2720,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",<p>Controllare le impostazioni del vassoio car...


In [7]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
#keep only the text

In [9]:
# Assuming you have a DataFrame named 'df' and you want to see all rows of the 'column_name' column
column_name = 'Text'
pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_df[column_name])
pd.reset_option('display.max_rows')  # Reset the option to its default value

0       <p>Ciao,</p> <p>la mia tastiera purtropppo non...
1       <p>Ciao,</p> <p>da oggi non vedo piu il link p...
2       <p>Ciao,</p> <p>Ho bisogno di accedere a IDEpr...
3       Ciao,<br />lui deve acceder all'appartamento A...
4       <p>Ciao,<br />purtroppo hanno fatto un pò casi...
5       <p>Ciao,</p> <p>Volevo sapere se riceviamo anc...
6       <p>Ciao Manuel,</p> <p>Grazie. Sarebbe possibi...
7                                           <p>Grazie</p>
8       <p>Hallo, ich bekomme immer einen Fehler dass ...
9                      <p>169 578 378</p> <p>nqpdezyw</p>
10      <p>Ciao,</p> <p>Per favore reattivare l'utente...
11      <p>Ciao</p> <p>Mi poete collegare per favore q...
12      <p>Ciao,</p> <p>la stampante Verkauf PCV on pr...
13      <p>Egregi Signori,</p> <p>Vi prego di programm...
14      <p>Hello,</p> <p>I have no acess to CRM app an...
15      <p>Hi,</p> <p>I need to order a new laptop for...
16      <p>Hi,</p> <p>I would like to ask if it is sti...
17      <p>Cia

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [11]:
def remove_html_tags(text):
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    
    # Remove HTML tags and extract only the text
    text = soup.get_text()
    
    # Remove any remaining special characters
    pattern = r'[^a-zA-Z0-9\s]'
    return re.sub(pattern, '', text)

# Apply the remove_html_tags function to the DataFrame columns containing HTML data
filtered_df['Text'] = filtered_df['Text'].apply(remove_html_tags)


C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\1070994409.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [12]:
column_name = 'Text'
pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_df[column_name])
pd.reset_option('display.max_rows')  # Reset the option to its default value

0       Ciao la mia tastiera purtropppo non funziona p...
1       Ciao da oggi non vedo piu il link per il teams...
2       Ciao Ho bisogno di accedere a IDEpro ma quando...
3       Ciaolui deve acceder allappartamento AgostiniB...
4       Ciaopurtroppo hanno fatto un p casinoPotete pe...
5       Ciao Volevo sapere se riceviamo ancora una not...
6       Ciao Manuel Grazie Sarebbe possibile avere una...
7                                                  Grazie
8       Hallo ich bekomme immer einen Fehler dass die ...
9                                    169 578 378 nqpdezyw
10      Ciao Per favore reattivare lutente TAEE Mikola...
11      Ciao Mi poete collegare per favore questa stam...
12      Ciao la stampante Verkauf PCV on prtserv01  bl...
13      Egregi Signori Vi prego di programmare il badg...
14      Hello I have no acess to CRM app and d3 smart ...
15      Hi I need to order a new laptop for me Could y...
16      Hi I would like to ask if it is still OK to or...
17      Ciaoco

In [13]:
filtered_df.to_csv('cleaned_data.csv', index=False)

In [14]:
df = pd.read_csv('cleaned_data.csv')
df

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la mia tastiera purtropppo non funziona p...
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",Ciao da oggi non vedo piu il link per il teams...
2,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",Ciao Ho bisogno di accedere a IDEpro ma quando...
3,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",Ciaolui deve acceder allappartamento AgostiniB...
4,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopurtroppo hanno fatto un p casinoPotete pe...
...,...,...,...
2717,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ora funziona grazie mille
2718,Netzwerk und Kameraeinstellung,"{""12"":""Standard Software: Operating systems, M...",Hallo ich habe keinen Zugriff auaf das Netzw...
2719,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciaoprego dare a Stefano un PC di scorta che p...
2720,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Controllare le impostazioni del vassoio carta ...


In [15]:
df = df.dropna()

# The df_cleaned DataFrame will contain only rows with no NaN values
df

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la mia tastiera purtropppo non funziona p...
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",Ciao da oggi non vedo piu il link per il teams...
2,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",Ciao Ho bisogno di accedere a IDEpro ma quando...
3,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",Ciaolui deve acceder allappartamento AgostiniB...
4,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopurtroppo hanno fatto un p casinoPotete pe...
...,...,...,...
2717,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ora funziona grazie mille
2718,Netzwerk und Kameraeinstellung,"{""12"":""Standard Software: Operating systems, M...",Hallo ich habe keinen Zugriff auaf das Netzw...
2719,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciaoprego dare a Stefano un PC di scorta che p...
2720,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Controllare le impostazioni del vassoio carta ...


In [16]:
df.to_csv('final_data.csv', index=False)

In [17]:
df = pd.read_csv('final_data.csv')
df

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la mia tastiera purtropppo non funziona p...
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",Ciao da oggi non vedo piu il link per il teams...
2,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",Ciao Ho bisogno di accedere a IDEpro ma quando...
3,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",Ciaolui deve acceder allappartamento AgostiniB...
4,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopurtroppo hanno fatto un p casinoPotete pe...
...,...,...,...
2584,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ora funziona grazie mille
2585,Netzwerk und Kameraeinstellung,"{""12"":""Standard Software: Operating systems, M...",Hallo ich habe keinen Zugriff auaf das Netzw...
2586,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciaoprego dare a Stefano un PC di scorta che p...
2587,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Controllare le impostazioni del vassoio carta ...


In [18]:
#DELETE CONFIDENTIAL MESSAGE

import pandas as pd
import re

# Create or load your DataFrame 'df' with columns 'Title', 'Topic', and 'Text'

# Define a regular expression pattern to match the confidential message
confidential_pattern = re.compile(r"(?i)THIS MESSAGE INCLUDING ANY ATTACHMENTS CONTAINS CONFIDENTIAL INFORMATION.*?PROHIBITED", re.DOTALL)

# Function to remove text containing the confidential message
def remove_confidential_text(text):
    return re.sub(confidential_pattern, '', text)

# Apply the function to the 'Text' column
df['Text'] = df['Text'].apply(remove_confidential_text)

# Display the modified DataFrame
print(df)


                                 Title  \
0                             Tastiera   
1     Outlook  Termine erstellen TEams   
2                 IDEpro Authenticator   
3      NOSAL MIROSLAV - Badge / ACCESS   
4                          Licenze CRM   
...                                ...   
2584                carattere mancante   
2585    Netzwerk und Kameraeinstellung   
2586          PC di scorta per Stefano   
2587          Stampante ufficio Italia   
2588               Computer più veloce   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"12":"Standard Software: Operating systems, M...   
3     {"16":"User identity management, Permissions, ...   
4     {"16":"User identity management, Permissions, ...   
...                                                 ...   
2584  {"12":"Standard Software: Operating systems, M...   
2585  {"12":"Standard S

In [19]:
import pandas as pd

desired_value = r'{"16":"User identity management, Permissions, Badge, Mail"}'

df_1 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_1

,Title,Topic,Text
3,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",Ciaolui deve acceder allappartamento AgostiniB...
4,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopurtroppo hanno fatto un p casinoPotete pe...
5,Password Windows,"{""16"":""User identity management, Permissions, ...",Ciao Volevo sapere se riceviamo ancora una not...
6,Password Windows,"{""16"":""User identity management, Permissions, ...",Ciao Manuel Grazie Sarebbe possibile avere una...
10,Reattivare utente EE DMS Mikolaj C.,"{""16"":""User identity management, Permissions, ...",Ciao Per favore reattivare lutente TAEE Mikola...
...,...,...,...
2544,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopotremmo per favore assegnare la licenza d...
2545,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ah upps sorry dovrebbe arrivare lunedi pensavo...
2566,indirizzo Mail engo.parts@engo-ice.com,"{""16"":""User identity management, Permissions, ...",Ciao Birgit ci ha creato il nuovo indirizzo ma...
2578,TechnoAlpin Cloud,"{""16"":""User identity management, Permissions, ...",I am selecting to reset the password but it do...


In [20]:
# Reset the index to start from 0
df_1 = df_1.reset_index(drop=True)
df_1

,Title,Topic,Text
0,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",Ciaolui deve acceder allappartamento AgostiniB...
1,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopurtroppo hanno fatto un p casinoPotete pe...
2,Password Windows,"{""16"":""User identity management, Permissions, ...",Ciao Volevo sapere se riceviamo ancora una not...
3,Password Windows,"{""16"":""User identity management, Permissions, ...",Ciao Manuel Grazie Sarebbe possibile avere una...
4,Reattivare utente EE DMS Mikolaj C.,"{""16"":""User identity management, Permissions, ...",Ciao Per favore reattivare lutente TAEE Mikola...
...,...,...,...
888,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopotremmo per favore assegnare la licenza d...
889,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ah upps sorry dovrebbe arrivare lunedi pensavo...
890,indirizzo Mail engo.parts@engo-ice.com,"{""16"":""User identity management, Permissions, ...",Ciao Birgit ci ha creato il nuovo indirizzo ma...
891,TechnoAlpin Cloud,"{""16"":""User identity management, Permissions, ...",I am selecting to reset the password but it do...


In [21]:
import pandas as pd

# Convert the 'Text' column to uppercase
df_1['Text'] = df_1['Text'].str.upper()

df_1

,Title,Topic,Text
0,NOSAL MIROSLAV - Badge / ACCESS,"{""16"":""User identity management, Permissions, ...",CIAOLUI DEVE ACCEDER ALLAPPARTAMENTO AGOSTINIB...
1,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPURTROPPO HANNO FATTO UN P CASINOPOTETE PE...
2,Password Windows,"{""16"":""User identity management, Permissions, ...",CIAO VOLEVO SAPERE SE RICEVIAMO ANCORA UNA NOT...
3,Password Windows,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL GRAZIE SAREBBE POSSIBILE AVERE UNA...
4,Reattivare utente EE DMS Mikolaj C.,"{""16"":""User identity management, Permissions, ...",CIAO PER FAVORE REATTIVARE LUTENTE TAEE MIKOLA...
...,...,...,...
888,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE LA LICENZA D...
889,Licenze CRM,"{""16"":""User identity management, Permissions, ...",AH UPPS SORRY DOVREBBE ARRIVARE LUNEDI PENSAVO...
890,indirizzo Mail engo.parts@engo-ice.com,"{""16"":""User identity management, Permissions, ...",CIAO BIRGIT CI HA CREATO IL NUOVO INDIRIZZO MA...
891,TechnoAlpin Cloud,"{""16"":""User identity management, Permissions, ...",I AM SELECTING TO RESET THE PASSWORD BUT IT DO...


In [22]:
import pandas as pd

desired_value = r'{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}'  # Note the 'r' before the string for a raw string

df_2 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_2

,Title,Topic,Text
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",Ciao da oggi non vedo piu il link per il teams...
2,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",Ciao Ho bisogno di accedere a IDEpro ma quando...
7,digitale Prüfprotokoll non funziona,"{""12"":""Standard Software: Operating systems, M...",Grazie
8,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",Hallo ich bekomme immer einen Fehler dass die ...
9,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",169 578 378 nqpdezyw
...,...,...,...
2575,Phising,"{""12"":""Standard Software: Operating systems, M...",Quando clicco nel outlook su Phising melden mi...
2581,Microsoft Teams,"{""12"":""Standard Software: Operating systems, M...",Ciao tutto il giorno che riscontro problemi c...
2583,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",Devo tradurre una presentazione su power point...
2584,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ora funziona grazie mille


In [23]:
# Reset the index to start from 0
df_2 = df_2.reset_index(drop=True)
df_2

,Title,Topic,Text
0,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",Ciao da oggi non vedo piu il link per il teams...
1,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",Ciao Ho bisogno di accedere a IDEpro ma quando...
2,digitale Prüfprotokoll non funziona,"{""12"":""Standard Software: Operating systems, M...",Grazie
3,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",Hallo ich bekomme immer einen Fehler dass die ...
4,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",169 578 378 nqpdezyw
...,...,...,...
806,Phising,"{""12"":""Standard Software: Operating systems, M...",Quando clicco nel outlook su Phising melden mi...
807,Microsoft Teams,"{""12"":""Standard Software: Operating systems, M...",Ciao tutto il giorno che riscontro problemi c...
808,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",Devo tradurre una presentazione su power point...
809,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ora funziona grazie mille


In [24]:
df_2['Text'] = df_2['Text'].str.upper()

df_2

,Title,Topic,Text
0,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...
1,IDEpro Authenticator,"{""12"":""Standard Software: Operating systems, M...",CIAO HO BISOGNO DI ACCEDERE A IDEPRO MA QUANDO...
2,digitale Prüfprotokoll non funziona,"{""12"":""Standard Software: Operating systems, M...",GRAZIE
3,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",HALLO ICH BEKOMME IMMER EINEN FEHLER DASS DIE ...
4,Adobe Lizenz,"{""12"":""Standard Software: Operating systems, M...",169 578 378 NQPDEZYW
...,...,...,...
806,Phising,"{""12"":""Standard Software: Operating systems, M...",QUANDO CLICCO NEL OUTLOOK SU PHISING MELDEN MI...
807,Microsoft Teams,"{""12"":""Standard Software: Operating systems, M...",CIAO TUTTO IL GIORNO CHE RISCONTRO PROBLEMI C...
808,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",DEVO TRADURRE UNA PRESENTAZIONE SU POWER POINT...
809,carattere mancante,"{""12"":""Standard Software: Operating systems, M...",ORA FUNZIONA GRAZIE MILLE


In [25]:
import pandas as pd

desired_value = r'{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}'

df_3 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_3

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la mia tastiera purtropppo non funziona p...
11,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao Mi poete collegare per favore questa stam...
12,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la stampante Verkauf PCV on prtserv01 bl...
15,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi I need to order a new laptop for me Could y...
16,new mobile for service technician,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi I would like to ask if it is still OK to or...
...,...,...,...
2580,Headset,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ich bin seit ein paar Tagen mit Martin Hofer z...
2582,Fixtelefon geht nicht,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hallo Fixtelefon geht nicht LG Niklas
2586,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciaoprego dare a Stefano un PC di scorta che p...
2587,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Controllare le impostazioni del vassoio carta ...


In [26]:
# Reset the index to start from 0
df_3 = df_3.reset_index(drop=True)
df_3

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la mia tastiera purtropppo non funziona p...
1,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao Mi poete collegare per favore questa stam...
2,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao la stampante Verkauf PCV on prtserv01 bl...
3,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi I need to order a new laptop for me Could y...
4,new mobile for service technician,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi I would like to ask if it is still OK to or...
...,...,...,...
880,Headset,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ich bin seit ein paar Tagen mit Martin Hofer z...
881,Fixtelefon geht nicht,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hallo Fixtelefon geht nicht LG Niklas
882,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciaoprego dare a Stefano un PC di scorta che p...
883,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Controllare le impostazioni del vassoio carta ...


In [27]:
df_3['Text'] = df_3['Text'].str.upper()

df_3

,Title,Topic,Text
0,Tastiera,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA MIA TASTIERA PURTROPPPO NON FUNZIONA P...
1,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI POETE COLLEGARE PER FAVORE QUESTA STAM...
2,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE VERKAUF PCV ON PRTSERV01 BL...
3,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...
4,new mobile for service technician,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I WOULD LIKE TO ASK IF IT IS STILL OK TO OR...
...,...,...,...
880,Headset,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ICH BIN SEIT EIN PAAR TAGEN MIT MARTIN HOFER Z...
881,Fixtelefon geht nicht,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO FIXTELEFON GEHT NICHT LG NIKLAS
882,PC di scorta per Stefano,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAOPREGO DARE A STEFANO UN PC DI SCORTA CHE P...
883,Stampante ufficio Italia,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CONTROLLARE LE IMPOSTAZIONI DEL VASSOIO CARTA ...


# MOST USED WORDS

# df_1
{"16":"User identity management, Permissions, Badge, Mail"}

In [28]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [29]:
import nltk
nltk.download('stopwords')

# Tokenize and remove stopwords
stop_words = set(stopwords.words('english'))
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize and remove stopwords for the 'Text' column
stop_words = set(stopwords.words('english'))
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in stop_words])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_1['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(100)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('per', 316), ('di', 308), ('grazie', 265), ('e', 247), ('il', 236), ('ciao', 231), ('non', 216), ('la', 182), ('che', 182), ('badge', 138), ('da', 124), ('mi', 118), ('bitte', 114), ('mail', 112), ('hallo', 106), ('come', 101), ('le', 99), ('die', 99), ('un', 98), ('al', 94), ('saluti', 93), ('technoalpin', 93), ('prego', 91), ('email', 89), ('danke', 81), ('favore', 80), ('una', 80), ('con', 80), ('von', 80), ('please', 79), ('39', 76), ('via', 73), ('und', 71), ('ich', 69), ('ha', 69), ('fr', 69), ('mille', 67), ('potete', 67), ('der', 65), ('anche', 65), ('password', 64), ('nicht', 61), ('2', 60), ('outlook', 58), ('hello', 58), ('1', 57), ('se', 56), ('ho', 56), ('0471', 53), ('sono', 51), ('del', 50), ('manuel', 50), ('hi', 50), ('licenza', 49), ('den', 49), ('vi', 47), ('si', 46), ('id', 44), ('ad', 43), ('das', 43), ('ta', 43), ('im', 42), ('agostini', 42), ('accesso', 41), ('nuovo', 40), ('buongiorno', 38), ('account', 38), ('0', 38), ('horst', 37), ('alle', 36), ('pi', 36), 

In [31]:
#It includes a custom list of words to remove

import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella']


# Tokenize and remove stopwords from the 'Text' column
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_1['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('badge', 138), ('mail', 112), ('email', 89), ('password', 64), ('outlook', 58), ('licenza', 49), ('accesso', 41), ('account', 38), ('accedere', 33), ('pc', 33), ('access', 30), ('diritti', 30), ('nav', 26), ('computer', 25), ('server', 25), ('vpn', 25), ('crm', 23), ('send', 21), ('sap', 21), ('mensa', 21), ('concur', 20), ('message', 20), ('porte', 19), ('laptop', 19), ('phone', 18), ('microsoft', 18), ('ufficio', 17), ('fino', 16), ('persone', 16), ('emails', 16)]


In [32]:
import pandas as pd

# Create keywords for each subset
keywords = {
    'df_1_email': ['message', 'mail', 'email', 'emails', 'account', 'sender', 'recipient',
                   'microsoft', 'outlook', 'password', 'attachments', 'send','licenza'],
    'df_2_rights': ['badge', 'accedere', 'diritti', 'access', 'porte', 'mensa', 'accesso'],
    'df_3_computer': ['pc', 'laptop', 'computer', 'phone', 'password'],
    'df_4_software': ['server', 'nav', 'vpn', 'crm', 'sap', 'concur']
}

# Initialize empty DataFrames for each subset
subset_dfs = {key: pd.DataFrame(columns=['Text']) for key in keywords.keys()}

# Iterate through each row in df_1 and assign it to the appropriate subset
for index, row in df_1.iterrows():
    text = row['Text']
    for subset_name, subset_keywords in keywords.items():
        if any(keyword in text.lower() for keyword in subset_keywords):
            subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)

# Now, you have four subsets in separate DataFrames
df_1_email = subset_dfs['df_1_email']
df_2_rights = subset_dfs['df_2_rights']
df_3_computer = subset_dfs['df_3_computer']
df_4_software = subset_dfs['df_4_software']

# You can access and work with each subset as needed
print("df_1_email:")
print(df_1_email)

print("\ndf_2_rights:")
print(df_2_rights)

print("\ndf_3_computer:")
print(df_3_computer)

print("\ndf_4_software:")
print(df_4_software)


C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\4151013874.py:20: FutureWarning: The frame.append method is d

df_1_email:
                                                  Text
0    CIAOPURTROPPO HANNO FATTO UN P CASINOPOTETE PE...
1    CIAO VOLEVO SAPERE SE RICEVIAMO ANCORA UNA NOT...
2    CIAO MANUEL GRAZIE SAREBBE POSSIBILE AVERE UNA...
3    CIAOPOSSIAMO PER FAVORE ASSEGNARE UNA LICENZA ...
4    HALLO BITTE MEINE EMAIL SIGNATUR VON TECHNOALP...
..                                                 ...
311                      153 359 970 PASSWORD BBKHGJUF
312  CIAO NO IN REALT MI SUCCEDE IN UFFICIO SENZA H...
313  CIAOPOTREMMO PER FAVORE ASSEGNARE LA LICENZA D...
314  CIAO BIRGIT CI HA CREATO IL NUOVO INDIRIZZO MA...
315  I AM SELECTING TO RESET THE PASSWORD BUT IT DO...

[316 rows x 1 columns]

df_2_rights:
                                                  Text
0    EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...
1    CIAO   MI SERVE UN BADGE PER MARTIN GAJDOS   C...
2    BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO B...
3    THIS MORNING I SET THE NEW PASSWORD ON MY COMP...
4    CIAO MANUE

## df_1_email

In [33]:
import pandas as pd

words_to_check = ['message', 'mail', 'email', 'emails', 'account', 'sender', 'recipient',
                   'microsoft', 'outlook', 'password', 'attachments', 'send','licenza']
# Initialize an empty DataFrame to store the matching rows
df_1_email = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_email = df_1_email.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_email = df_1_email.reset_index(drop=True)

df_1_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word

,Title,Topic,Text,tokens
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...,"[receiving, error, message, surface, opening, ..."
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...,"[karima, guerbaa, absent, until, september, 15..."
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...,"[would, like, retrieve, outlook, 2022, messages]"
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
...,...,...,...,...
545,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...,"[ciaopotremmo, licenza, bill, wang, jason, wu,..."
546,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...,"[ciaopotete, schgaguler, licenza, member, crmi..."
547,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...,"[ciaopotete, licenza, full, crm, florian, mayr..."
548,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE A DIETMAR MA...,"[ciaopotremmo, dietmar, mair, licenza, memberg..."


In [34]:
df_1_email

,Title,Topic,Text,tokens
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...,"[receiving, error, message, surface, opening, ..."
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...,"[karima, guerbaa, absent, until, september, 15..."
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...,"[would, like, retrieve, outlook, 2022, messages]"
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
...,...,...,...,...
545,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...,"[ciaopotremmo, licenza, bill, wang, jason, wu,..."
546,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...,"[ciaopotete, schgaguler, licenza, member, crmi..."
547,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...,"[ciaopotete, licenza, full, crm, florian, mayr..."
548,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE A DIETMAR MA...,"[ciaopotremmo, dietmar, mair, licenza, memberg..."


In [35]:
df_1_email = df_1_email.drop(columns='tokens')
df_1_email

,Title,Topic,Text
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...
...,...,...,...
545,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...
546,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...
547,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...
548,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE A DIETMAR MA...


# df_1_rights

In [36]:
#df_2_rights
import pandas as pd

words_to_check = ['badge', 'accedere', 'diritti', 'access', 'porte', 'mensa', 'accesso']
# Initialize an empty DataFrame to store the matching rows
df_1_rights = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_rights = df_1_rights.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_rights = df_1_rights.reset_index(drop=True)

df_1_rights

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(d

,Title,Topic,Text,tokens
0,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...,"[programmare, badge, 10147, segue, accesso, po..."
1,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...,"[badge, gajdos, stessi, accessi, altri, saldat..."
2,Autorizzazione Badge,"{""16"":""User identity management, Permissions, ...",BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO B...,"[avrei, abilitare, badge, accessi, autorizzazi..."
3,Appartamento HQ Agostini,"{""16"":""User identity management, Permissions, ...",CIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV...,"[abilitare, badge, nosal, miroslav, stano, jur..."
4,Badge n. 1031 Anja Höller,"{""16"":""User identity management, Permissions, ...",CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO P...,"[aver, impostato, tutto, anjavolevo, avvisarvi..."
...,...,...,...,...
370,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...,"[mysql, server, esterno, bloccato, taiotmysqld..."
371,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...,"[assegnato, support, dellit, engo, volevo, chi..."
372,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...,"[cortesemente, accesso, lettura, lorenzo, sart..."
373,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."


In [37]:
df_1_rights

,Title,Topic,Text,tokens
0,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...,"[programmare, badge, 10147, segue, accesso, po..."
1,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...,"[badge, gajdos, stessi, accessi, altri, saldat..."
2,Autorizzazione Badge,"{""16"":""User identity management, Permissions, ...",BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO B...,"[avrei, abilitare, badge, accessi, autorizzazi..."
3,Appartamento HQ Agostini,"{""16"":""User identity management, Permissions, ...",CIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV...,"[abilitare, badge, nosal, miroslav, stano, jur..."
4,Badge n. 1031 Anja Höller,"{""16"":""User identity management, Permissions, ...",CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO P...,"[aver, impostato, tutto, anjavolevo, avvisarvi..."
...,...,...,...,...
370,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...,"[mysql, server, esterno, bloccato, taiotmysqld..."
371,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...,"[assegnato, support, dellit, engo, volevo, chi..."
372,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...,"[cortesemente, accesso, lettura, lorenzo, sart..."
373,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."


In [38]:
df_1_rights = df_1_rights.drop(columns='tokens')
df_1_rights

,Title,Topic,Text
0,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...
1,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...
2,Autorizzazione Badge,"{""16"":""User identity management, Permissions, ...",BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO B...
3,Appartamento HQ Agostini,"{""16"":""User identity management, Permissions, ...",CIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV...
4,Badge n. 1031 Anja Höller,"{""16"":""User identity management, Permissions, ...",CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO P...
...,...,...,...
370,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...
371,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...
372,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...
373,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...


# df_1_computer

In [39]:
#df_3_computer
import pandas as pd

words_to_check = ['pc', 'laptop', 'computer', 'phone', 'password']
# Initialize an empty DataFrame to store the matching rows
df_1_computer = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_computer = df_1_computer.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_computer = df_1_computer.reset_index(drop=True)

df_1_computer

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_c

,Title,Topic,Text,tokens
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...,"[penso, yu, yang, cambiato, password, tempo, e..."
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...,"[ciaomi, mia, signatura, nelloutlook, dopo, ca..."
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...,"[ragazzi, vorrei, accedere, rete, intranet, vi..."
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...,"[access, sapconcur, cant, login, because, forg..."
...,...,...,...,...
128,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."
129,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...,"[servirebbe, vostro, admin, password, poter, i..."
130,Email account,"{""16"":""User identity management, Permissions, ...",HI MANUEL CAU YOU PLEASE SEND ME THE PASSWORD ...,"[cau, send, password, monashitechnoalpincomtha..."
131,Problema invio mail,"{""16"":""User identity management, Permissions, ...",153 359 970 PASSWORD BBKHGJUF,"[153, 359, 970, password, bbkhgjuf]"


In [40]:
df_1_computer

,Title,Topic,Text,tokens
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...,"[penso, yu, yang, cambiato, password, tempo, e..."
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...,"[ciaomi, mia, signatura, nelloutlook, dopo, ca..."
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...,"[ragazzi, vorrei, accedere, rete, intranet, vi..."
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...,"[access, sapconcur, cant, login, because, forg..."
...,...,...,...,...
128,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."
129,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...,"[servirebbe, vostro, admin, password, poter, i..."
130,Email account,"{""16"":""User identity management, Permissions, ...",HI MANUEL CAU YOU PLEASE SEND ME THE PASSWORD ...,"[cau, send, password, monashitechnoalpincomtha..."
131,Problema invio mail,"{""16"":""User identity management, Permissions, ...",153 359 970 PASSWORD BBKHGJUF,"[153, 359, 970, password, bbkhgjuf]"


In [41]:
df_1_computer = df_1_computer.drop(columns='tokens')
df_1_computer

,Title,Topic,Text
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...
...,...,...,...
128,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...
129,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...
130,Email account,"{""16"":""User identity management, Permissions, ...",HI MANUEL CAU YOU PLEASE SEND ME THE PASSWORD ...
131,Problema invio mail,"{""16"":""User identity management, Permissions, ...",153 359 970 PASSWORD BBKHGJUF


# df_1_software

In [42]:
#df_1_software
import pandas as pd

words_to_check = ['server', 'nav', 'vpn', 'crm', 'sap', 'concur']
# Initialize an empty DataFrame to store the matching rows
df_1_software = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_software = df_1_software.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_software = df_1_software.reset_index(drop=True)

df_1_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_softw

,Title,Topic,Text,tokens
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...,"[koennen, lserver, installieren, fuer, neue, k..."
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...,"[aber, koennen, server, finden, offnen]"
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...,"[contattato, stephane, detto, dato, diritti, q..."
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...,"[dopo, cambiamento, server, mandare, email, da..."
...,...,...,...,...
164,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...,"[impostare, lutente, andrej, mladenoski, utent..."
165,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...,"[dear, all, create, concur, account, employee,..."
166,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...,"[conto, sap, concur, stato, creato, antoine, p..."
167,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...,"[noted, access, sap, concur, our, newly, emplo..."


In [43]:
df_1_software

,Title,Topic,Text,tokens
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...,"[koennen, lserver, installieren, fuer, neue, k..."
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...,"[aber, koennen, server, finden, offnen]"
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...,"[contattato, stephane, detto, dato, diritti, q..."
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...,"[dopo, cambiamento, server, mandare, email, da..."
...,...,...,...,...
164,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...,"[impostare, lutente, andrej, mladenoski, utent..."
165,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...,"[dear, all, create, concur, account, employee,..."
166,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...,"[conto, sap, concur, stato, creato, antoine, p..."
167,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...,"[noted, access, sap, concur, our, newly, emplo..."


In [44]:
df_1_software = df_1_software.drop(columns='tokens')
df_1_software

,Title,Topic,Text
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...
...,...,...,...
164,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...
165,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...
166,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...
167,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...


# df_2
{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}

In [45]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella', 'problem', 'poi', 'digitale','also','du', 'problema', 'io', 
                    'es', 'here', 'pdf', 'devo', 'as','marco','41', 'provato', 'install', 
                    'tutto', 'eine','va', 'sempre', 'open','work','do','programma','one']

# Tokenize and remove stopwords from the 'Text' column
df_2['tokens'] = df_2['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_2['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('pc', 59), ('email', 57), ('outlook', 54), ('teams', 44), ('installare', 40), ('pw', 34), ('mail', 33), ('nav', 32), ('laptop', 31), ('password', 29), ('office', 25), ('computer', 24), ('accedere', 23), ('citrix', 23), ('vpn', 22), ('accesso', 21), ('message', 20), ('prfprotokoll', 19), ('vdi', 18), ('server', 18), ('microsoft', 17), ('acrobat', 17), ('passwort', 16), ('edge', 16), ('app', 15), ('emails', 15), ('vedo', 14), ('account', 14), ('messaggio', 14), ('br', 14)]


# df_2_pc

In [46]:
#df_2_pc
import pandas as pd

words_to_check = ['pc', 'computer', 'accedere', 'pw', 'password',
                  'laptop', 'passwort', 'pwd', 'login', 'accesso']

# Initialize an empty DataFrame to store the matching rows
df_2_pc = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_pc = df_2_pc.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_pc = df_2_pc.reset_index(drop=True)

df_2_pc

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA

,Title,Topic,Text,tokens
0,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
1,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...,"[nach, einem, neustart, meines, pc, scheint, p..."
2,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...,"[pc, nav, italie, austria, doesnt, best]"
3,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...,"[servirebbe, linstallazione, windows, 10, pc, ..."
4,problemi linceza Mapedit,"{""12"":""Standard Software: Operating systems, M...",CIAO ARASHDEEP SCUSAMI MA HO RIAVVIATO IL PC E...,"[arashdeep, scusami, riavviato, pc, ed, ora, b..."
...,...,...,...,...
271,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
272,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."
273,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...,"[chiedere, voi, ticketing, system, technoalpin..."
274,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...,"[installare, accesso, vpncitrix, laptop, urgen..."


In [47]:
df_2_pc

,Title,Topic,Text,tokens
0,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
1,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...,"[nach, einem, neustart, meines, pc, scheint, p..."
2,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...,"[pc, nav, italie, austria, doesnt, best]"
3,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...,"[servirebbe, linstallazione, windows, 10, pc, ..."
4,problemi linceza Mapedit,"{""12"":""Standard Software: Operating systems, M...",CIAO ARASHDEEP SCUSAMI MA HO RIAVVIATO IL PC E...,"[arashdeep, scusami, riavviato, pc, ed, ora, b..."
...,...,...,...,...
271,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
272,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."
273,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...,"[chiedere, voi, ticketing, system, technoalpin..."
274,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...,"[installare, accesso, vpncitrix, laptop, urgen..."


In [48]:
df_2_pc = df_2_pc.drop(columns='tokens')
df_2_pc

,Title,Topic,Text
0,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...
1,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...
2,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...
3,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...
4,problemi linceza Mapedit,"{""12"":""Standard Software: Operating systems, M...",CIAO ARASHDEEP SCUSAMI MA HO RIAVVIATO IL PC E...
...,...,...,...
271,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...
272,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...
273,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...
274,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...


# df_2_email

In [49]:
#df_2_email
import pandas as pd

words_to_check = ['message', 'email', 'outlook', 'emails', 
                  'mail', 'accedere', 'pw', 'password', 'passwort', 
                  'recipient', 'attachments', 'pwd', 'microsoft', 'office', 'app', 'login', 'accesso']


# Initialize an empty DataFrame to store the matching rows
df_2_email = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_email = df_2_email.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_email = df_2_email.reset_index(drop=True)

df_2_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mas

,Title,Topic,Text,tokens
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...,"[absender, whitelist, setzen, wichtig, fehlerm..."
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...,"[press, cirtix, remote, desktop, symbol, conne..."
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...,"[ultimamente, abbiamo, problemi, grossi, mail,..."
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...,"[helloafter, changed, password, cant, connect,..."
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...,"[linda, zhang, accessing, italian, system, cit..."
...,...,...,...,...
461,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
462,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."
463,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...,"[chiedere, voi, ticketing, system, technoalpin..."
464,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...,"[installare, accesso, vpncitrix, laptop, urgen..."


In [50]:
df_2_email

,Title,Topic,Text,tokens
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...,"[absender, whitelist, setzen, wichtig, fehlerm..."
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...,"[press, cirtix, remote, desktop, symbol, conne..."
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...,"[ultimamente, abbiamo, problemi, grossi, mail,..."
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...,"[helloafter, changed, password, cant, connect,..."
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...,"[linda, zhang, accessing, italian, system, cit..."
...,...,...,...,...
461,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
462,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."
463,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...,"[chiedere, voi, ticketing, system, technoalpin..."
464,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...,"[installare, accesso, vpncitrix, laptop, urgen..."


In [51]:
df_2_email = df_2_email.drop(columns = 'tokens')
df_2_email

,Title,Topic,Text
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...
...,...,...,...
461,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...
462,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...
463,Accesso http://web01/erticket/scp/login.php pe...,"{""12"":""Standard Software: Operating systems, M...",DEVO CHIEDERE A VOI LACCESSO AL TICKETING SYST...
464,Installare accesso VPN/CITRIX sul mio Laptop,"{""12"":""Standard Software: Operating systems, M...",INSTALLARE ACCESSO VPNCITRIX SUL MIO LAPTOP UR...


# df_2_access

In [52]:
#df_2_access
import pandas as pd

words_to_check = ['teams', 'login', 'accesso', 'accedere', 'pw', 'password']


# Initialize an empty DataFrame to store the matching rows
df_2_access = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_access = df_2_access.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_access = df_2_access.reset_index(drop=True)

df_2_access

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[

,Title,Topic,Text,tokens
0,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...,"[vedo, piu, link, teams, meeting, creo, appunt..."
1,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...,"[collegamento, tra, programmi, sopracitati, fi..."
2,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...,"[nostro, partner, finlandia, manu, laari, manu..."
3,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...,"[qualche, settimana, camera, computer, home, o..."
4,Blocco Outlook e Teams,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL PRESENTE TICKET PERCH NON RIESCO...,"[presente, outlook, teams, compare, messaggio,..."
...,...,...,...,...
187,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...,"[trying, adobe, pro, will, let, had, old, inst..."
188,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...,"[accesso, vedo, 10, cartelle, drive, epowertra..."
189,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...,"[lid, rimane, password, aveva, timeout, qui, q..."
190,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...,"[update, windows, viewer, 545, 489, 361passwor..."


In [53]:
df_2_access

,Title,Topic,Text,tokens
0,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...,"[vedo, piu, link, teams, meeting, creo, appunt..."
1,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...,"[collegamento, tra, programmi, sopracitati, fi..."
2,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...,"[nostro, partner, finlandia, manu, laari, manu..."
3,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...,"[qualche, settimana, camera, computer, home, o..."
4,Blocco Outlook e Teams,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL PRESENTE TICKET PERCH NON RIESCO...,"[presente, outlook, teams, compare, messaggio,..."
...,...,...,...,...
187,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...,"[trying, adobe, pro, will, let, had, old, inst..."
188,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...,"[accesso, vedo, 10, cartelle, drive, epowertra..."
189,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...,"[lid, rimane, password, aveva, timeout, qui, q..."
190,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...,"[update, windows, viewer, 545, 489, 361passwor..."


In [54]:
df_2_access = df_2_access.drop(columns = 'tokens')
df_2_access

,Title,Topic,Text
0,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...
1,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...
2,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...
3,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...
4,Blocco Outlook e Teams,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL PRESENTE TICKET PERCH NON RIESCO...
...,...,...,...
187,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...
188,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...
189,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...
190,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...


# df_2_software

In [55]:
#df_2_software
import pandas as pd

words_to_check = ['installare', 'nav', 'citrix', 'vpn', 'edge', 'acrobat', 'prfprotokoll', 'vdi', 'server']

# Initialize an empty DataFrame to store the matching rows
df_2_software = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_software = df_2_software.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_software = df_2_software.reset_index(drop=True)

df_2_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_s

,Title,Topic,Text,tokens
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...,"[installare, word, excel, ed, outlook, macchin..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...,"[ciaomi, installare, pack, solid, edge, sp7gra..."
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...,"[riapro, questione, acrobat, reader, 64bit, im..."
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...,"[discusso, voce, department, installare, softw..."
...,...,...,...,...
205,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...,"[migrazione, server, sembra, andata, bene, err..."
206,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
207,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...,"[servono, servizio, trisoft, deve, mandare, em..."
208,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...,"[our, technican, unable, recive, mail, his, la..."


In [56]:
df_2_software

,Title,Topic,Text,tokens
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...,"[installare, word, excel, ed, outlook, macchin..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...,"[ciaomi, installare, pack, solid, edge, sp7gra..."
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...,"[riapro, questione, acrobat, reader, 64bit, im..."
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...,"[discusso, voce, department, installare, softw..."
...,...,...,...,...
205,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...,"[migrazione, server, sembra, andata, bene, err..."
206,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
207,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...,"[servono, servizio, trisoft, deve, mandare, em..."
208,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...,"[our, technican, unable, recive, mail, his, la..."


In [57]:
df_2_software = df_2_software.drop(columns = 'tokens')
df_2_software

,Title,Topic,Text
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...
...,...,...,...
205,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...
206,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...
207,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...
208,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...


# df_3
{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}

In [58]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella', 'problem', 'poi', 'digitale','also','du', 'problema', 'io', 
                    'es', 'here', 'pdf', 'devo', 'as','marco','41', 'provato', 'install', 
                    'tutto', 'eine','va', 'sempre', 'open','work','do','programma','one', 
                    'verificare','message','installato', '3306', 'high','ufficio', 'problemi', 
                    'uno','prone','t41', 'netprodclientsofficewlan','abbiamo', 'avere', 
                    'vulnerability', 'fa', 'francesco', 'sarebbe', 'installare', 'purtroppo', 'avete',
                    '75', 'fatto', 'multiple', 'dos', 'funktioniert','vecchio', 'domani','qualche',
                    'numero', 'piu', 'volta']

# Tokenize and remove stopwords from the 'Text' column
df_3['tokens'] = df_3['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_3['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('stampante', 104), ('laptop', 100), ('pc', 94), ('telefono', 45), ('docking', 44), ('stampare', 33), ('station', 30), ('monitor', 30), ('printer', 30), ('drucker', 28), ('fisso', 27), ('rete', 27), ('zebra', 25), ('computer', 25), ('cavo', 24), ('stampa', 22), ('schermo', 21), ('tastiera', 20), ('software', 20), ('pw', 20), ('mia', 18), ('cellulare', 18), ('usb', 18), ('mail', 18), ('password', 18), ('ein', 18), ('office', 18), ('possiamo', 18), ('ogni', 18), ('secondo', 17)]


# df_3_printer

In [59]:
#df_3_printer
import pandas as pd

words_to_check = ['stampante', 'stampare', 'printer', 'drucker', 'zebra', 'stampa']

# Initialize an empty DataFrame to store the matching rows
df_3_printer = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_printer = df_3_printer.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_printer = df_3_printer.reset_index(drop=True)

df_3_printer

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.appe

,Title,Topic,Text,tokens
0,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI POETE COLLEGARE PER FAVORE QUESTA STAM...,"[poete, collegare, stampante]"
1,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE VERKAUF PCV ON PRTSERV01 BL...,"[stampante, verkauf, pcv, prtserv01, bloccata,..."
2,stampante service secondo piano sopra,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA STAMPANTE SERVICE SECONDO PIANO NON ULTIMA ...,"[stampante, secondo, piano, ultima, lavori, d,..."
3,Stampante zebra non stampa,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE ZEBRA NON STAMPA DA NAVISION,"[stampante, zebra, stampa, navision]"
4,Cambio ufficio,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO IO E JOHANNES CI SIAMO SPOSTATI NELLUFFIC...,"[johannes, siamo, spostati, nellufficio, era, ..."
...,...,...,...,...
326,Problema accesso cartella \\Controller-TX30,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI NON RIUSCIAMO AD ACCEDERE ALLA CA...,"[riusciamo, accedere, controllertx30, quindi, ..."
327,Installation SECURE PRINTER (GANG VK),"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO INSTALLARE LA POSSIBILITA DI STAMPA...,"[possibilita, stampare, secure, printer, secon..."
328,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE HP2891F8 NON FUNZIONA GRAZIE,"[stampante, hp2891f8]"
329,RE: Ticket #513035 Assigned to you,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO BASTA CHE RIAVVIA IL PC DA DOVE VUOLE ...,"[basta, riavvia, pc, dove, vuole, stampare, za..."


In [60]:
df_3_printer

,Title,Topic,Text,tokens
0,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI POETE COLLEGARE PER FAVORE QUESTA STAM...,"[poete, collegare, stampante]"
1,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE VERKAUF PCV ON PRTSERV01 BL...,"[stampante, verkauf, pcv, prtserv01, bloccata,..."
2,stampante service secondo piano sopra,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA STAMPANTE SERVICE SECONDO PIANO NON ULTIMA ...,"[stampante, secondo, piano, ultima, lavori, d,..."
3,Stampante zebra non stampa,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE ZEBRA NON STAMPA DA NAVISION,"[stampante, zebra, stampa, navision]"
4,Cambio ufficio,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO IO E JOHANNES CI SIAMO SPOSTATI NELLUFFIC...,"[johannes, siamo, spostati, nellufficio, era, ..."
...,...,...,...,...
326,Problema accesso cartella \\Controller-TX30,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI NON RIUSCIAMO AD ACCEDERE ALLA CA...,"[riusciamo, accedere, controllertx30, quindi, ..."
327,Installation SECURE PRINTER (GANG VK),"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO INSTALLARE LA POSSIBILITA DI STAMPA...,"[possibilita, stampare, secure, printer, secon..."
328,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE HP2891F8 NON FUNZIONA GRAZIE,"[stampante, hp2891f8]"
329,RE: Ticket #513035 Assigned to you,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO BASTA CHE RIAVVIA IL PC DA DOVE VUOLE ...,"[basta, riavvia, pc, dove, vuole, stampare, za..."


In [61]:
df_3_printer = df_3_printer.drop(columns = 'tokens')
df_3_printer

,Title,Topic,Text
0,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI POETE COLLEGARE PER FAVORE QUESTA STAM...
1,stampante bloccata,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE VERKAUF PCV ON PRTSERV01 BL...
2,stampante service secondo piano sopra,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA STAMPANTE SERVICE SECONDO PIANO NON ULTIMA ...
3,Stampante zebra non stampa,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE ZEBRA NON STAMPA DA NAVISION
4,Cambio ufficio,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO IO E JOHANNES CI SIAMO SPOSTATI NELLUFFIC...
...,...,...,...
326,Problema accesso cartella \\Controller-TX30,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI NON RIUSCIAMO AD ACCEDERE ALLA CA...
327,Installation SECURE PRINTER (GANG VK),"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO INSTALLARE LA POSSIBILITA DI STAMPA...
328,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE HP2891F8 NON FUNZIONA GRAZIE
329,RE: Ticket #513035 Assigned to you,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO BASTA CHE RIAVVIA IL PC DA DOVE VUOLE ...


# df_3_docking

In [62]:
#df_3_docking
import pandas as pd
words_to_check = ['docking', 'cavo', 'rete', 'lan', 'station']

# Initialize an empty DataFrame to store the matching rows
df_3_docking = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_docking = df_3_docking.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_docking = df_3_docking.reset_index(drop=True)

df_3_docking

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.appe

,Title,Topic,Text,tokens
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...,"[manca, adattatore, usb, lan, ne, dovrei, riti..."
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...,"[vorrei, sostituire, pc, fisso, portatile, doc..."
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...,"[larrivo, teresa, buffa, 0409, spostarmi, sar,..."
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...,"[ask, about, thing, happened, morning, went, o..."
...,...,...,...,...
160,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...,"[dati, 1726129643, x85i1w2t, cose, impostare, ..."
161,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...,"[collegamento, tra, laptopdocking, station, mo..."
162,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...,"[controllato, docking, station, dopo, laggiorn..."
163,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...,"[accendo, pc, docking, station, andare, tastie..."


In [63]:
df_3_docking

,Title,Topic,Text,tokens
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...,"[manca, adattatore, usb, lan, ne, dovrei, riti..."
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...,"[vorrei, sostituire, pc, fisso, portatile, doc..."
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...,"[larrivo, teresa, buffa, 0409, spostarmi, sar,..."
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...,"[ask, about, thing, happened, morning, went, o..."
...,...,...,...,...
160,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...,"[dati, 1726129643, x85i1w2t, cose, impostare, ..."
161,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...,"[collegamento, tra, laptopdocking, station, mo..."
162,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...,"[controllato, docking, station, dopo, laggiorn..."
163,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...,"[accendo, pc, docking, station, andare, tastie..."


In [64]:
df_3_docking = df_3_docking.drop(columns = 'tokens')
df_3_docking

,Title,Topic,Text
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...
...,...,...,...
160,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...
161,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...
162,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...
163,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...


# df_3_tcp

In [65]:
#df_3_tcp
import pandas as pd
words_to_check = ['tcp', 'windows']

# Initialize an empty DataFrame to store the matching rows
df_3_tcp = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_tcp = df_3_tcp.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_tcp = df_3_tcp.reset_index(drop=True)

df_3_tcp

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\589532251.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_tcp = df_3_tcp.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\589532251.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_tcp = df_3_tcp.append(df_3[word_mask])


,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [66]:
df_3_tcp

,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [67]:
df_3_tcp = df_3_tcp.drop(columns= 'tokens')
df_3_tcp

,Title,Topic,Text
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...


# df_3_db 

In [68]:
#df_3_db
import pandas as pd
words_to_check = ['oracle', 'mysql', 'windows', 'mariadb']

# Initialize an empty DataFrame to store the matching rows
df_3_db = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_db = df_3_db.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_db = df_3_db.reset_index(drop=True)

df_3_db

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])


,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [69]:
df_3_db

,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [70]:
df_3_db = df_3_db.drop(columns = 'tokens')
df_3_db

,Title,Topic,Text
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...


# df_3_pc

In [71]:
#df_3_pc
import pandas as pd
words_to_check = ['laptop', 'pc', 'schermo', 'fisso', 'tastiera', 'monitor', 
                  'computer', 'cavo', 'pw', 'usb', 'telefono']


# Initialize an empty DataFrame to store the matching rows
df_3_pc = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_pc = df_3_pc.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_pc = df_3_pc.reset_index(drop=True)

df_3_pc

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA

,Title,Topic,Text,tokens
0,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...,"[order, laptop, give, some, advice, which, sui..."
1,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...,"[seit, kurzem, aufgefallen, sich, mein, laptop..."
2,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
3,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...,"[valutare, sostituzione, quanto, lento, sia, a..."
4,Teams und Kopfhörer,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HABE DEN LAPTOP MIT DEN AIRPODS VERBUNDEN ABER...,"[laptop, airpods, verbunden, aber, hre, teamsa..."
...,...,...,...,...
385,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...,"[collegarmi, telefono, fisso, telefono, 125]"
386,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI,"[telefono, fisso]"
387,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...,"[buon, pomeriggio, effetturare, chiamateverso,..."
388,Telefono Cordless,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO ATTUALMENTE MI STATO FORNITO UN T...,"[attualmente, stato, fornito, telefono, fisso,..."


In [72]:
df_3_pc

,Title,Topic,Text,tokens
0,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...,"[order, laptop, give, some, advice, which, sui..."
1,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...,"[seit, kurzem, aufgefallen, sich, mein, laptop..."
2,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
3,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...,"[valutare, sostituzione, quanto, lento, sia, a..."
4,Teams und Kopfhörer,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HABE DEN LAPTOP MIT DEN AIRPODS VERBUNDEN ABER...,"[laptop, airpods, verbunden, aber, hre, teamsa..."
...,...,...,...,...
385,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...,"[collegarmi, telefono, fisso, telefono, 125]"
386,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI,"[telefono, fisso]"
387,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...,"[buon, pomeriggio, effetturare, chiamateverso,..."
388,Telefono Cordless,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO ATTUALMENTE MI STATO FORNITO UN T...,"[attualmente, stato, fornito, telefono, fisso,..."


In [73]:
df_3_pc = df_3_pc.drop(columns = 'tokens')
df_3_pc

,Title,Topic,Text
0,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...
1,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...
2,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...
3,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...
4,Teams und Kopfhörer,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HABE DEN LAPTOP MIT DEN AIRPODS VERBUNDEN ABER...
...,...,...,...
385,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...
386,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI
387,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...
388,Telefono Cordless,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO ATTUALMENTE MI STATO FORNITO UN T...


# df_3_vulnerabilities

In [74]:
#df_3_vulnerabilities
import pandas as pd
words_to_check = ['vulnerabilities']

# Initialize an empty DataFrame to store the matching rows
df_3_vulnerabilities = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_vulnerabilities = df_3_vulnerabilities.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_vulnerabilities = df_3_vulnerabilities.reset_index(drop=True)
df_3_vulnerabilities= df_3_vulnerabilities.drop(columns = 'tokens')
df_3_vulnerabilities

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\155574697.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_vulnerabilities = df_3_vulnerabilities.append(df_3[word_mask])


,Title,Topic,Text


In [75]:
df_3_vulnerabilities

,Title,Topic,Text


# df_3_email

In [76]:
#df_3_email
import pandas as pd
words_to_check = ['mail']

# Initialize an empty DataFrame to store the matching rows
df_3_email = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_email = df_3_email.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_email = df_3_email.reset_index(drop=True)
df_3_email= df_3_email.drop(columns = 'tokens')
df_3_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\750117258.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_email = df_3_email.append(df_3[word_mask])


,Title,Topic,Text
0,Numero telefonico,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON GIORNO IL MIO NUMERO TELEFONICE SEMBRA NO...
1,Nr telefono,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE POTRESTE CAMBIARE IL NUMERO DI TELEFONO ...
2,Info for ticket 582224,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO FRANCESCO THX NOW WORK FOR MY I WILL MA...
3,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I CALLED HIM AND SENT A MAIL WITH PHOTO THE SC...
4,Toner stampante ACS 1247109,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO FINITO IL TONER DELLA STAMPANTE XE...
5,Cellulare e ProCall a Laura,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO PROCEDERE CON LA RICHIESTA A LAURA ...
6,Neues ThinkPad,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO BITTE DAS THINKPAD FR BIANCA ZANONA EIN...
7,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO PER LUTILIZZO DI MAIL E TELEFONATE ...
8,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DANIELE VEDI LO SCREEN DELLA MAIL CON LAP...
9,e-mail salvata su stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTRESTE SALVARE LA MIA EMAIL NELLA STAM...


In [77]:
df_3_email

,Title,Topic,Text
0,Numero telefonico,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON GIORNO IL MIO NUMERO TELEFONICE SEMBRA NO...
1,Nr telefono,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE POTRESTE CAMBIARE IL NUMERO DI TELEFONO ...
2,Info for ticket 582224,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO FRANCESCO THX NOW WORK FOR MY I WILL MA...
3,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I CALLED HIM AND SENT A MAIL WITH PHOTO THE SC...
4,Toner stampante ACS 1247109,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO FINITO IL TONER DELLA STAMPANTE XE...
5,Cellulare e ProCall a Laura,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO PROCEDERE CON LA RICHIESTA A LAURA ...
6,Neues ThinkPad,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO BITTE DAS THINKPAD FR BIANCA ZANONA EIN...
7,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO PER LUTILIZZO DI MAIL E TELEFONATE ...
8,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DANIELE VEDI LO SCREEN DELLA MAIL CON LAP...
9,e-mail salvata su stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTRESTE SALVARE LA MIA EMAIL NELLA STAM...


# df_3_software

In [78]:
#df_3_software
import pandas as pd
words_to_check = ['software', 'server']

# Initialize an empty DataFrame to store the matching rows
df_3_software = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_software = df_3_software.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_software = df_3_software.reset_index(drop=True)
df_3_software= df_3_software.drop(columns = 'tokens')
df_3_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\916185259.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_software = df_3_software.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_2124\916185259.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_software = df_3_software.append(df_3[word_mask])


,Title,Topic,Text
0,PC running slow.,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY PC HAS BEEN RUNNING VERY SLOW FOR THE PAST ...
1,Monitor non si collega con il portatile,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO STAMATTINA NON RIESCO A COLLEGARMI COL MO...
2,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE SE POSSIBILE CHIEDEREI CHE FRANCESCO O M...
3,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PER ME VA BENE PER DOMANI VIENE CONCLUSO LA CO...
4,Neues Notebook SPS-Programmierer TAEE Peter Št...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WIE VON HANNES SIMONINI ANGEFORDERT BESTELLE...
5,Sostituzione PC,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CHIEDO GENTILMENTE LA SOSTITUZIONE ...
6,PC C 238 startet nicht mehr,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NAVISION TEAMVIEWER DIE WEETECH SOFTWARE MSSEN...
7,PC Showroom,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAOPOTRESTE PER FAVORE METTERE A DISPOSIZIONE...
8,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...
9,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...


In [79]:
df_3_software

,Title,Topic,Text
0,PC running slow.,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY PC HAS BEEN RUNNING VERY SLOW FOR THE PAST ...
1,Monitor non si collega con il portatile,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO STAMATTINA NON RIESCO A COLLEGARMI COL MO...
2,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE SE POSSIBILE CHIEDEREI CHE FRANCESCO O M...
3,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PER ME VA BENE PER DOMANI VIENE CONCLUSO LA CO...
4,Neues Notebook SPS-Programmierer TAEE Peter Št...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WIE VON HANNES SIMONINI ANGEFORDERT BESTELLE...
5,Sostituzione PC,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CHIEDO GENTILMENTE LA SOSTITUZIONE ...
6,PC C 238 startet nicht mehr,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NAVISION TEAMVIEWER DIE WEETECH SOFTWARE MSSEN...
7,PC Showroom,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAOPOTRESTE PER FAVORE METTERE A DISPOSIZIONE...
8,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...
9,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...


# SUMMARIZATION

# Creation of blocks 

## df_1_email

In [80]:
pip install --upgrade requests


Note: you may need to restart the kernel to use updated packages.


In [81]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [95]:
#specify number of rows in each chunk
n=2
#split DataFrame into chunks
list_df = [df_1_email[i:i+n] for i in range(0,len(df_1_email),n)]
    #print(list_df)

In [96]:
len(list_df)

275

In [97]:
list_df[274]

,Title,Topic,Text
548,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE A DIETMAR MA...
549,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE LA LICENZA D...


In [98]:
def put_together():
    list1=[]
    for index in range(0,274):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [99]:
summ_block = put_together()
summ_block

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

['I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS\xa0 I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED\xa0 ERROR CODE 80090016HI KARIMA GUERBAA\xa0IS ABSENT UNTIL SEPTEMBER 15\xa0SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS',
 'HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT\xa0 THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN\xa0SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE ',
 'HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE I

In [101]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 274):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [102]:
blocks_to_skip = [4,5,47,110,140,168,169,173,174,176,195,204,234,241,242,244,248,250]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

In [103]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

['I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS\xa0 I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED\xa0 ERROR CODE 80090016HI KARIMA GUERBAA\xa0IS ABSENT UNTIL SEPTEMBER 15\xa0SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS',
 'HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT\xa0 THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN\xa0SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE ',
 'HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE I

In [90]:
!pip install torch
!pip install transformers

In [105]:
import pickle

In [107]:
picklefile = "df_1_email_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block_test = pickle.load(pkl_rb_obj)
print(summ_block_test)

[[{'summary_text': "There is an error message on Surface when opening Outlook Team. Karima Guerba is absent until September 15th. He will send his e-mail in French to his contact in France in order to put a message of absence on his Outlook team's website."}], [{'summary_text': 'The Outlook 2022 is still not closing. There is a new problem with the mailbox showing more messages than in the inbox. The person responsible will check the inbox in order to send it to the Outlook 2022 email in advance. It is not working properly yet. '}], [{'summary_text': "The Outlook is still not closing. There is a new problem with the e-mail. Daniela's employment ends tomorrow, but her company email will still receive the messages we need for accountancy.   It's not clear how it works in other departments. "}], [{'summary_text': "Andres Schwingershack sent a mesage to the office of Lorsquoffery and received a late reply from Manuel. He is a member of the Board of Directors of the Lawyer's Association of 

# df_1_rights

In [115]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_rights[i:i+n] for i in range(0,len(df_1_rights),n)]
    #print(list_df)

In [116]:
len(list_df)

63

In [117]:
list_df[62]

,Title,Topic,Text
372,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...
373,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...
374,Dare accesso di lettura a cartelle,"{""16"":""User identity management, Permissions, ...",BUONGIORNO VI CHIEDO DI DARE ACCESSO A DAVID M...


In [120]:
def put_together():
    list1=[]
    for index in range(0,62):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [121]:
summ_block = put_together()
summ_block

Result 0:
EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADGE N 10147 COME SEGUE ACCESSO SOLO DALLE PORTE DELLOFFICINA NIENTE UFFICIO DA LUNEDI A VENERDI DALLE ORE 530 FINO ALLE 1900 GRAZIE SALUTI HORSTCIAO   MI SERVE UN BADGE PER MARTIN GAJDOS   CON GLI STESSI ACCESSI COME GLI ALTRI SALDATORI   GRAZIE BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO BADGE A TUTTI GLI ACCESSI AUTORIZZAZIONI COME MICHELE VALBUSA IERI NELLA FILIALE DI VIA SIEMENS NON ENTRAVO DALLE PORTECIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV E STANO JURAJ ALLINGRESSO DELLAPPARTAMENTO QUI NEL HQ GRAZIE HANNES  CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO PER ANJAVOLEVO AVVISARVI CHE AVEVAMO GI UN BADGE PER LEI N 10230  VI RIDIAMO IL N 1031 CHE NON CI SERVE LO CONSEGNA LORENZO OGGI A MEZZOGIORNO GRAZIE EGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10149 PERCH OGGI TERMINA IL TIROCINO DI IOLANDO IULIANO GRAZIE SALUTI HORST

Result 1:
EGREGI SIGNORI VI PREGO DI ATTIVARE IL NUOVO BADGE N 10009 DI ANDREY COME SEGUE INGRESSO

['EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADGE N 10147 COME SEGUE ACCESSO SOLO DALLE PORTE DELLOFFICINA NIENTE UFFICIO DA LUNEDI A VENERDI DALLE ORE 530 FINO ALLE 1900 GRAZIE SALUTI HORSTCIAO\xa0  MI SERVE UN BADGE PER\xa0MARTIN GAJDOS\xa0\xa0 CON GLI STESSI ACCESSI COME GLI ALTRI SALDATORI\xa0  GRAZIE\xa0BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO BADGE A TUTTI GLI ACCESSI AUTORIZZAZIONI COME MICHELE VALBUSA IERI NELLA FILIALE DI VIA SIEMENS NON ENTRAVO DALLE PORTECIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV E STANO JURAJ ALLINGRESSO DELLAPPARTAMENTO QUI NEL HQ GRAZIE HANNES \xa0CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO PER ANJAVOLEVO AVVISARVI CHE AVEVAMO GI UN BADGE PER LEI N 10230  VI RIDIAMO IL N 1031 CHE NON CI SERVE LO CONSEGNA LORENZO OGGI A MEZZOGIORNO GRAZIE EGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10149 PERCH OGGI TERMINA IL TIROCINO DI IOLANDO IULIANO GRAZIE SALUTI HORST',
 'EGREGI SIGNORI VI PREGO DI ATTIVARE IL NUOVO BADGE N 10009 DI ANDREY COME SEGUE IN

In [123]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 62):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [124]:
blocks_to_skip = [12,16,39,43,51,58,61]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADGE N 10147 COME SEGUE ACCESSO SOLO DALLE PORTE DELLOFFICINA NIENTE UFFICIO DA LUNEDI A VENERDI DALLE ORE 530 FINO ALLE 1900 GRAZIE SALUTI HORSTCIAO   MI SERVE UN BADGE PER MARTIN GAJDOS   CON GLI STESSI ACCESSI COME GLI ALTRI SALDATORI   GRAZIE BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO BADGE A TUTTI GLI ACCESSI AUTORIZZAZIONI COME MICHELE VALBUSA IERI NELLA FILIALE DI VIA SIEMENS NON ENTRAVO DALLE PORTECIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV E STANO JURAJ ALLINGRESSO DELLAPPARTAMENTO QUI NEL HQ GRAZIE HANNES  CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO PER ANJAVOLEVO AVVISARVI CHE AVEVAMO GI UN BADGE PER LEI N 10230  VI RIDIAMO IL N 1031 CHE NON CI SERVE LO CONSEGNA LORENZO OGGI A MEZZOGIORNO GRAZIE EGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10149 PERCH OGGI TERMINA IL TIROCINO DI IOLANDO IULIANO GRAZIE SALUTI HORST

Result 1:
EGREGI SIGNORI VI PREGO DI ATTIVARE IL NUOVO BADGE N 10009 DI ANDREY COME SEGUE INGRESSO

In [125]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADGE N 10147 COME SEGUE ACCESSO SOLO DALLE PORTE DELLOFFICINA NIENTE UFFICIO DA LUNEDI A VENERDI DALLE ORE 530 FINO ALLE 1900 GRAZIE SALUTI HORSTCIAO   MI SERVE UN BADGE PER MARTIN GAJDOS   CON GLI STESSI ACCESSI COME GLI ALTRI SALDATORI   GRAZIE BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO BADGE A TUTTI GLI ACCESSI AUTORIZZAZIONI COME MICHELE VALBUSA IERI NELLA FILIALE DI VIA SIEMENS NON ENTRAVO DALLE PORTECIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV E STANO JURAJ ALLINGRESSO DELLAPPARTAMENTO QUI NEL HQ GRAZIE HANNES  CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO PER ANJAVOLEVO AVVISARVI CHE AVEVAMO GI UN BADGE PER LEI N 10230  VI RIDIAMO IL N 1031 CHE NON CI SERVE LO CONSEGNA LORENZO OGGI A MEZZOGIORNO GRAZIE EGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10149 PERCH OGGI TERMINA IL TIROCINO DI IOLANDO IULIANO GRAZIE SALUTI HORST

Result 1:
EGREGI SIGNORI VI PREGO DI ATTIVARE IL NUOVO BADGE N 10009 DI ANDREY COME SEGUE INGRESSO

['EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADGE N 10147 COME SEGUE ACCESSO SOLO DALLE PORTE DELLOFFICINA NIENTE UFFICIO DA LUNEDI A VENERDI DALLE ORE 530 FINO ALLE 1900 GRAZIE SALUTI HORSTCIAO\xa0  MI SERVE UN BADGE PER\xa0MARTIN GAJDOS\xa0\xa0 CON GLI STESSI ACCESSI COME GLI ALTRI SALDATORI\xa0  GRAZIE\xa0BUONGIORNO AVREI BISOGNO DI ABILITARE IL MIO BADGE A TUTTI GLI ACCESSI AUTORIZZAZIONI COME MICHELE VALBUSA IERI NELLA FILIALE DI VIA SIEMENS NON ENTRAVO DALLE PORTECIAO PREGO ABILITARE I BADGE DI NOSAL MIROSLAV E STANO JURAJ ALLINGRESSO DELLAPPARTAMENTO QUI NEL HQ GRAZIE HANNES \xa0CIAO A TUTTI GRAZIE PER AVER IMPOSTATO TUTTO PER ANJAVOLEVO AVVISARVI CHE AVEVAMO GI UN BADGE PER LEI N 10230  VI RIDIAMO IL N 1031 CHE NON CI SERVE LO CONSEGNA LORENZO OGGI A MEZZOGIORNO GRAZIE EGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10149 PERCH OGGI TERMINA IL TIROCINO DI IOLANDO IULIANO GRAZIE SALUTI HORST',
 'EGREGI SIGNORI VI PREGO DI ATTIVARE IL NUOVO BADGE N 10009 DI ANDREY COME SEGUE IN

In [129]:
picklefile = "df_1_rights_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block_test = pickle.load(pkl_rb_obj)
print(summ_block_test)

[[{'summary_text': 'E.G. SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10147 is a GRAZIE SALUTI HORSTCIAO and IL MIO BADGE is a TUTTI GLI ACCESSI AUTORIZZAZIONI. '}], [{'summary_text': 'GRAZIE SALUTI HORSTEGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10004 PERCH MESSNER LUKAS AVEVA IERI IL SUO ULTIMO GIORNO DI LAVORO GRAZie SALUTi HORTPREGO FARE NUOVO PER MICHELE MARSONET.'}], [{'summary_text': 'EGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10150 PERCH ALEX GASSER TERMINA OGGI IL SUO TIROCINO IN ANTICIPO GRAZIE SALUTI HORSTCIAO RAGAZZI. '}], [{'summary_text': 'Clemens Pintt, ACCESSO GARAGE, ABILITARE BADGECIAO VI PREGO DI CREARE, UN BADGE PER JONSTON DARREN IN CASA FINO 010923 MENSA PAGATA PROFILO, IL BADGE N 10050 PERCH OGGI TERMINA, IL TIROCINIO DI GAMPER SIMON GRAZIE SALUTI HORST PREGO DISATTIVARE, IL Badge ADESSO PERCH HA LAVORATO SOLO FINO ALLE 1200, IL GOODGE A CASA POTETE PREPARARE UNO'}], [{'summary_text': 'CIAO PREGO VI CHIEDERE DI PREPARARE UN BADGE PER FELIX DILGER PER 170

# df_1_computer


In [132]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_computer[i:i+n] for i in range(0,len(df_1_computer),n)]
    #print(list_df)

In [133]:
len(list_df)

23

In [134]:
list_df[22]

,Title,Topic,Text
132,TechnoAlpin Cloud,"{""16"":""User identity management, Permissions, ...",I AM SELECTING TO RESET THE PASSWORD BUT IT DO...


In [142]:
def put_together():
    list1=[]
    for index in range(0,22):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [143]:
summ_block = put_together()
summ_block

Result 0:
CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA PASSWORD IN TEMPO ED  BLOCCATA POTRESTI SBLOCCARE IL PC MFL0228 PER FAVORE GRAZIECIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUTLOOK CHE DOPO IL CAMBIO PC NON C PIGRZCIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRANET PER VISUALIZZARE MATERIALI COME MANUALI PRODOTTI ECC COME FACCIO 2 VORREI ACCEDERE A PROTIME COME FACCIO HO CERCATO MA NON VEDO IL PROGRAMMA SUL PC 3 MI HANNO DI CHIEDERVI DI ABILITARE IL MIO BADGE PER ACCEDERE ALLA PRODUZIONE IN VIA SIEMENSI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO NON RIESCO PI A CONNETTERMI A INTERNET CON IL MIO PC HO AVUTO QUESTO PROBLEMA QUALCHE SETTIMANA FA POTRESTI PER FAVORE CONTROLLARLO IL PRIMA POSSIBILE SALUTI HANS

Result 1:
HALLO BITTE DEN ORDNER  GPRODUKTIONPRODUKTIONSLEITUNG FR   PC C247ALPINL

['CIAO PENSO CHE\xa0LE YU YANG NON HA CAMBIATO LA PASSWORD IN TEMPO ED  BLOCCATA POTRESTI SBLOCCARE IL PC MFL0228\xa0PER FAVORE GRAZIECIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUTLOOK CHE DOPO IL CAMBIO PC NON C PIGRZCIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRANET PER VISUALIZZARE MATERIALI COME MANUALI PRODOTTI ECC COME FACCIO 2 VORREI ACCEDERE A PROTIME COME FACCIO HO CERCATO MA NON VEDO IL PROGRAMMA SUL PC 3 MI HANNO DI CHIEDERVI DI ABILITARE IL MIO BADGE PER ACCEDERE ALLA PRODUZIONE IN VIA SIEMENSI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO NON RIESCO PI A CONNETTERMI A INTERNET CON IL MIO PC HO AVUTO QUESTO PROBLEMA QUALCHE SETTIMANA FA POTRESTI PER FAVORE CONTROLLARLO IL PRIMA POSSIBILE SALUTI HANS',
 'HALLO BITTE DEN ORDNER\xa0\xa0GPRODUKTIONPRODUKTIONSLEITUNG FR\xa0 \xa0PC C247ALP

In [144]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 22):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [145]:
blocks_to_skip = [2,4,12]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA PASSWORD IN TEMPO ED  BLOCCATA POTRESTI SBLOCCARE IL PC MFL0228 PER FAVORE GRAZIECIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUTLOOK CHE DOPO IL CAMBIO PC NON C PIGRZCIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRANET PER VISUALIZZARE MATERIALI COME MANUALI PRODOTTI ECC COME FACCIO 2 VORREI ACCEDERE A PROTIME COME FACCIO HO CERCATO MA NON VEDO IL PROGRAMMA SUL PC 3 MI HANNO DI CHIEDERVI DI ABILITARE IL MIO BADGE PER ACCEDERE ALLA PRODUZIONE IN VIA SIEMENSI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO NON RIESCO PI A CONNETTERMI A INTERNET CON IL MIO PC HO AVUTO QUESTO PROBLEMA QUALCHE SETTIMANA FA POTRESTI PER FAVORE CONTROLLARLO IL PRIMA POSSIBILE SALUTI HANS

Result 1:
HALLO BITTE DEN ORDNER  GPRODUKTIONPRODUKTIONSLEITUNG FR   PC C247ALPINL

In [146]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA PASSWORD IN TEMPO ED  BLOCCATA POTRESTI SBLOCCARE IL PC MFL0228 PER FAVORE GRAZIECIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUTLOOK CHE DOPO IL CAMBIO PC NON C PIGRZCIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRANET PER VISUALIZZARE MATERIALI COME MANUALI PRODOTTI ECC COME FACCIO 2 VORREI ACCEDERE A PROTIME COME FACCIO HO CERCATO MA NON VEDO IL PROGRAMMA SUL PC 3 MI HANNO DI CHIEDERVI DI ABILITARE IL MIO BADGE PER ACCEDERE ALLA PRODUZIONE IN VIA SIEMENSI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO NON RIESCO PI A CONNETTERMI A INTERNET CON IL MIO PC HO AVUTO QUESTO PROBLEMA QUALCHE SETTIMANA FA POTRESTI PER FAVORE CONTROLLARLO IL PRIMA POSSIBILE SALUTI HANS

Result 1:
HALLO BITTE DEN ORDNER  GPRODUKTIONPRODUKTIONSLEITUNG FR   PC C247ALPINL

['CIAO PENSO CHE\xa0LE YU YANG NON HA CAMBIATO LA PASSWORD IN TEMPO ED  BLOCCATA POTRESTI SBLOCCARE IL PC MFL0228\xa0PER FAVORE GRAZIECIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUTLOOK CHE DOPO IL CAMBIO PC NON C PIGRZCIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRANET PER VISUALIZZARE MATERIALI COME MANUALI PRODOTTI ECC COME FACCIO 2 VORREI ACCEDERE A PROTIME COME FACCIO HO CERCATO MA NON VEDO IL PROGRAMMA SUL PC 3 MI HANNO DI CHIEDERVI DI ABILITARE IL MIO BADGE PER ACCEDERE ALLA PRODUZIONE IN VIA SIEMENSI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO NON RIESCO PI A CONNETTERMI A INTERNET CON IL MIO PC HO AVUTO QUESTO PROBLEMA QUALCHE SETTIMANA FA POTRESTI PER FAVORE CONTROLLARLO IL PRIMA POSSIBILE SALUTI HANS',
 'HALLO BITTE DEN ORDNER\xa0\xa0GPRODUKTIONPRODUKTIONSLEITUNG FR\xa0 \xa0PC C247ALP

In [151]:
picklefile = "df_1_computer_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': "A new PC is not connected to the internet and I can't log in to my SAPCONCUR because I forgot my password. I have a new PC but can't connect it to the Internet because it's not connected with the Internet.   "}], [{'summary_text': 'HALLO BITTE DEN ORDNER: GPRODUKTION PRODUKTIONSLEITUNG FR, PC C247ALPIN LOCAL, FREIGEBEN NUR LESERECHTE DANKECIAO, HO CONTATTATO, STEPHANE, GRAZIEDANIELE MICHEL, FATTO LAGGIORNAMENTO, CHIESTO DAL, POI HA SPENTO E RIACCESO PERO ADESSO C QUESTO COSA DEVE FARENON CI SONO NELLA CARTELLA SPAM. '}], [{'summary_text': "There will be a new technician in France on May 15th. Her PC will be MFL0197, the same rightsprofile as SEBASTIEN COQUET's PC C235. It's not possible to connect DMS or NAVHLLO to the SVN server."}], [{'summary_text': 'All TAFR COLLABORATORS have signed a document in agreement with Human Resources Society to prevent the use of personal mobiles. The LAPTOP app does not work on the Laptop, so Daniele needs to find a solution for the 

# df_1_software

In [154]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_software[i:i+n] for i in range(0,len(df_1_software),n)]
    #print(list_df)

In [155]:
len(list_df)

29

In [156]:
list_df[28]

,Title,Topic,Text
168,Sap Concur Login,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL MI SPIACE DISTURBARTI MA TI CONFER...


In [161]:
def put_together():
    list1=[]
    for index in range(0,28):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [162]:
summ_block = put_together()
summ_block

Result 0:
HALLO  KOENNEN SIE BITTE LSERVER INSTALLIEREN FUER NEUE KOLEGIN LENKA SEVCIKOVA  SIE WIRD EIN TEAMVIEWER SCHICKEN  UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE KOENNEN SIE DAS AUCH SCHAUEN   DANKE ZUZANAOK ABER WIR KOENNEN NICHT SERVER FINDEN UND OFFNENI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABCIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE MI HA DATO I DIRITTI PER QUEI SERVERIO NON LI VEDO ANCORA DEVO FARE QUALCOSA SUL MIO PCDOPO IL CAMBIAMENTO SERVER SU TA NON  PI POSSIBILE MANDARE EMAIL DALLINDIRIZZO INFOEPOWERTRACCOM MA IN AUTOMATICO UTILIZZA LINDIRIZZO DELLUSER CHE STA SCRIVENDO  SALVANDO NELLUSCITA OUTLOOK DELLA PERSONA  NECESSERIO RIPRISTINARE   GRAZIE HELLO  ITS IMPOSSIBLE FOR ME TO CONNECT TO MN OR S SERVER IN BUILDING PRODUCTION  VIA SIEMENS BY WIFI IN VIA AGOSTINI ITS OK   MY TEAMVIEWER BR VINCENT BAUDOUIN

Result 1:
CIAO VI PREGO DI FARE 2 ACCOUNT PER I NUOVI COLLEGHI CHE INIZIERANNO LA PROSSIMA SETTIMANA LUNED  1 JOHANNES RIEDERI DIRITTI LACCES

['HALLO\xa0 KOENNEN SIE BITTE LSERVER INSTALLIEREN FUER NEUE KOLEGIN LENKA SEVCIKOVA\xa0 SIE WIRD EIN TEAMVIEWER SCHICKEN\xa0 UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE KOENNEN SIE DAS AUCH SCHAUEN \xa0 DANKE ZUZANAOK ABER WIR KOENNEN NICHT SERVER FINDEN UND\xa0OFFNENI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABCIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE MI HA DATO I DIRITTI PER QUEI SERVERIO NON LI VEDO ANCORA DEVO FARE QUALCOSA SUL MIO PCDOPO IL CAMBIAMENTO SERVER SU TA NON  PI POSSIBILE MANDARE EMAIL DALLINDIRIZZO INFOEPOWERTRACCOM MA IN AUTOMATICO UTILIZZA LINDIRIZZO DELLUSER CHE STA SCRIVENDO  SALVANDO NELLUSCITA OUTLOOK DELLA PERSONA  NECESSERIO RIPRISTINARE\xa0  GRAZIE\xa0HELLO\xa0 ITS IMPOSSIBLE FOR ME TO CONNECT TO MN OR S SERVER IN BUILDING PRODUCTION  VIA SIEMENS BY WIFI IN VIA AGOSTINI ITS OK   MY TEAMVIEWER BR VINCENT BAUDOUIN',
 'CIAO VI PREGO DI FARE 2 ACCOUNT PER I NUOVI COLLEGHI CHE INIZIERANNO LA PROSSIMA SETTIMANA LUNED\xa0 1 JOHANNES RIEDERI DI

In [163]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 28):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [164]:
blocks_to_skip = [3,4,12]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
HALLO  KOENNEN SIE BITTE LSERVER INSTALLIEREN FUER NEUE KOLEGIN LENKA SEVCIKOVA  SIE WIRD EIN TEAMVIEWER SCHICKEN  UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE KOENNEN SIE DAS AUCH SCHAUEN   DANKE ZUZANAOK ABER WIR KOENNEN NICHT SERVER FINDEN UND OFFNENI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABCIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE MI HA DATO I DIRITTI PER QUEI SERVERIO NON LI VEDO ANCORA DEVO FARE QUALCOSA SUL MIO PCDOPO IL CAMBIAMENTO SERVER SU TA NON  PI POSSIBILE MANDARE EMAIL DALLINDIRIZZO INFOEPOWERTRACCOM MA IN AUTOMATICO UTILIZZA LINDIRIZZO DELLUSER CHE STA SCRIVENDO  SALVANDO NELLUSCITA OUTLOOK DELLA PERSONA  NECESSERIO RIPRISTINARE   GRAZIE HELLO  ITS IMPOSSIBLE FOR ME TO CONNECT TO MN OR S SERVER IN BUILDING PRODUCTION  VIA SIEMENS BY WIFI IN VIA AGOSTINI ITS OK   MY TEAMVIEWER BR VINCENT BAUDOUIN

Result 1:
CIAO VI PREGO DI FARE 2 ACCOUNT PER I NUOVI COLLEGHI CHE INIZIERANNO LA PROSSIMA SETTIMANA LUNED  1 JOHANNES RIEDERI DIRITTI LACCES

In [165]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
HALLO  KOENNEN SIE BITTE LSERVER INSTALLIEREN FUER NEUE KOLEGIN LENKA SEVCIKOVA  SIE WIRD EIN TEAMVIEWER SCHICKEN  UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE KOENNEN SIE DAS AUCH SCHAUEN   DANKE ZUZANAOK ABER WIR KOENNEN NICHT SERVER FINDEN UND OFFNENI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABCIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE MI HA DATO I DIRITTI PER QUEI SERVERIO NON LI VEDO ANCORA DEVO FARE QUALCOSA SUL MIO PCDOPO IL CAMBIAMENTO SERVER SU TA NON  PI POSSIBILE MANDARE EMAIL DALLINDIRIZZO INFOEPOWERTRACCOM MA IN AUTOMATICO UTILIZZA LINDIRIZZO DELLUSER CHE STA SCRIVENDO  SALVANDO NELLUSCITA OUTLOOK DELLA PERSONA  NECESSERIO RIPRISTINARE   GRAZIE HELLO  ITS IMPOSSIBLE FOR ME TO CONNECT TO MN OR S SERVER IN BUILDING PRODUCTION  VIA SIEMENS BY WIFI IN VIA AGOSTINI ITS OK   MY TEAMVIEWER BR VINCENT BAUDOUIN

Result 1:
CIAO VI PREGO DI FARE 2 ACCOUNT PER I NUOVI COLLEGHI CHE INIZIERANNO LA PROSSIMA SETTIMANA LUNED  1 JOHANNES RIEDERI DIRITTI LACCES

['HALLO\xa0 KOENNEN SIE BITTE LSERVER INSTALLIEREN FUER NEUE KOLEGIN LENKA SEVCIKOVA\xa0 SIE WIRD EIN TEAMVIEWER SCHICKEN\xa0 UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE KOENNEN SIE DAS AUCH SCHAUEN \xa0 DANKE ZUZANAOK ABER WIR KOENNEN NICHT SERVER FINDEN UND\xa0OFFNENI HAVE A NEW PC BUT I CANT CONNECT SERVERS TAFR ET TABCIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE MI HA DATO I DIRITTI PER QUEI SERVERIO NON LI VEDO ANCORA DEVO FARE QUALCOSA SUL MIO PCDOPO IL CAMBIAMENTO SERVER SU TA NON  PI POSSIBILE MANDARE EMAIL DALLINDIRIZZO INFOEPOWERTRACCOM MA IN AUTOMATICO UTILIZZA LINDIRIZZO DELLUSER CHE STA SCRIVENDO  SALVANDO NELLUSCITA OUTLOOK DELLA PERSONA  NECESSERIO RIPRISTINARE\xa0  GRAZIE\xa0HELLO\xa0 ITS IMPOSSIBLE FOR ME TO CONNECT TO MN OR S SERVER IN BUILDING PRODUCTION  VIA SIEMENS BY WIFI IN VIA AGOSTINI ITS OK   MY TEAMVIEWER BR VINCENT BAUDOUIN',
 'CIAO VI PREGO DI FARE 2 ACCOUNT PER I NUOVI COLLEGHI CHE INIZIERANNO LA PROSSIMA SETTIMANA LUNED\xa0 1 JOHANNES RIEDERI DI

In [169]:
picklefile = "df_1_software_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': 'HALLO, KOENEN SIE BITTE, FUER NEUE, LENKA SEVCIKOVA, SIE WIRD EIN TEAMVIEWER SCHICKEN, UND DRUCKER FUNKCIONIERT ABER SCANER GEHT NICHT BITTE and DANKE ZUZANAOK ABER WIR.'}], [{'summary_text': 'There are 2 accounts to choose from: JOHANNES RIEDERI, DIRITTI LACCESSO SERVER EMAILSEGNATURA and LUKAS MESSNER. IM MISSING ACCESS TO THE CARQUEFOU SERVER MFS MPLEASE, I need to work. '}], [{'summary_text': "I don't have an access to the servers. I am looking for information on how to access them. I also need to contact the service provider.    The information on the services I need can be found on the following websites: \xa0http://www.servicom.com,  "}], [{'summary_text': 'Laccasso has set up a MySpace server. He has also set it up in UFFICIO SENZA HOTSPOT. His NAV licence is still not active, so he needs to find a solution for that. He needs to solve the issue with his NAV licence.'}], [{'summary_text': "I am still having problems getting access to the network. I can't chan

# df_2_pc

### df_3_software

In [ ]:
#specify number of rows in each chunk
n=3
#split DataFrame into chunks
list_df = [df_3_software[i:i+n] for i in range(0,len(df_3_software),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[11]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,12):
        res = ''.join(map(str,list_df[index]['Text']))
        print('Result:\n',res,'\n')
        list1.append(res)
    return list1

In [ ]:
summ_block = put_together()
summ_block

In [ ]:
import pickle

In [109]:
picklefile = "df_3_software_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': 'My PC has been running very slow for the past few weeks because of a software update. The Internet connection is also not working properly. I would like to know if there is anything you can do to help me.   i.a. 39 349 785 7499. '}], [{'summary_text': 'Paraforma IL Software is a tool for installing software on a PC. It is available in Italian, Spanish, Italian, German, Polish and Italian. It can be downloaded from the following sites: iTunes, Amazon, Google Play, Bittigte Software, Microsoft, and Play Store. '}], [{'summary_text': 'Naviision is a teamviewer, a software developer, a printer and a software installer. The software is available in English, German, Finnish, Italian and Spanish. It is available on the following platforms: Windows, Mac OS X, Android, iOS and Windows Phone.'}], [{'summary_text': 'Gli scanner needs a USB RS845 CONVERTER and a 24V power supply. He will send him SCHEMATICS soon.    i.e. the program is written in C without PYTHON and has his ow